In [11]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import requests
import pandas as pd
import pickle
import os

In [3]:
chaves = {
    'a': 1,
    'b': 2
}

def soma(a, b):
    return a + b

In [5]:
chaves['a']

1

In [6]:
import pendulum

In [19]:
import pendulum
data_hora_atual = pendulum.now('America/Sao_Paulo').to_iso8601_string()
data_hora_atual = pendulum.parse(data_hora_atual)
data_hora_busca = data_hora_atual.subtract(hours=7)
data_hora_busca_formatada = data_hora_busca.format('YYYY_MM_DD_HH_mm_ss')
data = f'extracao_data_{data_hora_busca_formatada}'


In [20]:
data

'extracao_data_2024_07_02_15_05_38'

In [2]:
soma(**chaves)

3

In [5]:
spark = SparkSession.builder.appName('DEPARA').getOrCreate()

24/07/02 19:45:37 WARN Utils: Your hostname, rodrigo-nitroan51554 resolves to a loopback address: 127.0.1.1; using 192.168.3.6 instead (on interface wlp8s0)
24/07/02 19:45:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/02 19:45:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df_estatisticas_videos = spark.read.json(
    '../datalake/bronze/cities_skylines/extracao_data_2024_06_29/estatisticas_canais_brasileiros/req_estatisticas_canais_brasileiros.json')

In [9]:
df_estatisticas_videos.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- customUrl: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- localized: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- publishedAt: string (nullable = true)
 |    |    |    |-- thumbnails: struct (nullable = true)
 |    |    |    |    |-- default: struct (nullable = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    

In [14]:
df_estatisticas_videos.select(
    'data_extracao',
    F.explode('items')
).printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- col: struct (nullable = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- customUrl: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- localized: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |-- publishedAt: string (nullable = true)
 |    |    |-- thumbnails: struct (nullable = true)
 |    |    |    |-- default: struct (nullable = true)
 |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |-- width: long (nullable = true)
 |    |    |    |-- high: struct (nullable = true)
 |    |    |    |    |-- height: long (nullable = true)
 |  

In [ ]:
def obter_hora(dataframe: DataFrame):
    dataframe = dataframe.withColumn('TURNO_EXTRACAO', F.when((F.col('HORA') >= 0) & (F.col('HORA') < 6), 'Madrugada')
                .when((F.col('HORA') >= 6) & (F.col('HORA') < 12), 'Manhã')
                .when((F.col('HORA') >= 12) & (F.col('HORA') < 18), 'Tarde')
                .otherwise('Noite'))

In [34]:
from pyspark.sql import functions as F

df_estatisticas_videos.select(
    'data_extracao',
    F.explode('items').alias('item')
).withColumn('HORA', F.hour('data_extracao')) \
    .select(
        F.col('data_extracao').alias('DATA_EXTRACAO'),
        F.col('TURNO_EXTRACAO'),
        F.col('HORA'),
        F.col('item.id').alias('ID'),
        F.col('item.snippet.title').alias('NM_CANAL'),
        F.col('item.statistics.subscriberCount').alias('TOTAL_INSCRITOS'),
        F.col('item.statistics.videoCount').alias('TOTAL_VIDEO_PUBLICADO'),
        F.col('item.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
).show()

+-------------------+--------------+----+--------------------+--------------------+---------------+---------------------+-------------------+
|      DATA_EXTRACAO|TURNO_EXTRACAO|HORA|                  ID|            NM_CANAL|TOTAL_INSCRITOS|TOTAL_VIDEO_PUBLICADO|TOTAL_VISUALIZACOES|
+-------------------+--------------+----+--------------------+--------------------+---------------+---------------------+-------------------+
|2024-06-29 23:34:29|         Noite|  23|UCTdehZL8aTm70QB6...|          DangerZone|             79|                   58|               3541|
|2024-06-29 23:34:29|         Noite|  23|UCsFvpmOu04Tyf9gs...|            JVR ZIKA|            225|                    4|                312|
|2024-06-29 23:34:29|         Noite|  23|UCOCp_GcNLExKx-Ox...|Bandeira Quadricu...|            305|                  117|              77521|
|2024-06-29 23:34:29|         Noite|  23|UC3OZQD5qIIt27gu_...|              Rc MTA|             87|                   16|               2297|
|2024-